In [9]:
# Read insurance ticker
import pandas as pd

insurance_df = pd.read_csv("../data/raw/insurance_hose_ticket_name.csv")
insurance_tickers = insurance_df["Mã CK"].tolist()
insurance_tickers

['BIC', 'BMI', 'BVH', 'MIG', 'PGI']

In [17]:
# Get company leader
from vnstock import Company
company = Company(symbol='BMI', source='VCI')
company.officers(filter_by='working')

,id,officer_name,officer_position,position_short_name,update_date,officer_own_percent,quantity
0,3,Nguyễn Thế Năng,Phó Tổng Giám đốc,Phó TGĐ,2023-03-21,1.900000e-04,20778
1,13,Châu Quang Linh,Phó Tổng Giám đốc,Phó TGĐ,2025-08-18,8.010000e-05,10628
2,8,Vũ Anh Tuấn,Thành viên Hội đồng Quản trị/Tổng Giám đốc,TV HĐQT/TGĐ,2025-08-18,4.000000e-05,5304
3,14,Sái Văn Hưng,Kế toán trưởng,Kế toán trưởng,2025-08-18,5.800000e-06,771
4,15,Phạm Minh Tuân,Phó Tổng Giám đốc,Phó TGĐ,2025-08-18,2.000000e-07,28
5,7,Nicolas De Nazelle,Thành viên Ban kiểm soát,TV BKS,2025-04-26,0.000000e+00,0
6,9,Trần Văn Tá,Thành viên Hội đồng Quản trị,TV HĐQT,2025-08-18,0.000000e+00,0
7,10,Đinh Việt Tùng,Thành viên Hội đồng Quản trị,TV HĐQT,2025-08-18,0.000000e+00,0
8,11,Lê Việt Thành,Thành viên Hội đồng Quản trị,TV HĐQT,2025-08-18,0.000000e+00,0
9,12,Bùi Thị Thu Thanh,Thành viên Hội đồng Quản trị,TV HĐQT,2025-08-18,0.000000e+00,0


In [31]:
# Get the data for insurance tickers
import pandas as pd
import numpy as np
from vnstock import Finance, Quote
from datetime import datetime, timedelta

# Get history data for insurance tickers
# for ticker in insurance_tickers:
# print(f"Processing {ticker}...")
# quote = Quote(symbol=ticker, source='VCI')
quote = Quote(symbol='BMI', source='VCI')
df_hist = quote.history(start='2014-11-26', end=datetime.now().strftime('%Y-%m-%d'), interval='1W')
df_hist_close = df_hist[['time', 'close', 'volume']]

df_hist_close = df_hist_close.sort_values('time')

# 2. Calculate the difference between the current date and the previous date
df_hist_close['days_diff'] = df_hist_close['time'].diff()

# 3. Filter rows where the difference is NOT 7 days (ignore the first row which will be NaT)
# We look for > 7 days. 
# Note: It is rare to have < 7 days in weekly data unless there are duplicates.
gaps = df_hist_close[df_hist_close['days_diff'] > pd.Timedelta(days=7)]

if gaps.empty:
    print("✅ Data is continuous! No missing weeks found.")
else:
    print(f"⚠️ Found {len(gaps)} gaps. Here are the weeks following a gap:")
    print(gaps[['time', 'days_diff']])

✅ Data is continuous! No missing weeks found.


# Get weekly close date to store in csv file for 5 insurance ticker

In [32]:
import pandas as pd
import time
from vnstock import Quote
from datetime import datetime

# 1. Configuration
# Top 5 Insurance stocks on HOSE
tickers = ['BIC', 'BMI', 'BVH', 'MIG', 'PGI']
start_date = '2014-11-26'
end_date = '2025-11-23'
output_file = '../data/raw/insurance_stocks_weekly_10y.csv'

all_data = [] # List to store dataframes

print(f"Fetching data from {start_date} to {end_date}...\n")

# 2. Loop through each ticker
for ticker in tickers:
    try:
        print(f"Processing {ticker}...")
        
        # Initialize Quote
        quote = Quote(symbol=ticker, source='VCI')
        
        # Get Data
        df = quote.history(start=start_date, end=end_date, interval='1W')
        
        # Basic Validation
        if df is None or df.empty:
            print(f"⚠️ No data found for {ticker}")
            continue

        # --- CLEANING STEPS (From our previous discussion) ---
        
        # A. Ensure datetime
        df['time'] = pd.to_datetime(df['time'])
        
        # B. Filter strict start date (Fixing the VCI 'batch' issue)
        df = df[df['time'] >= pd.to_datetime(start_date)]
        
        # C. Select columns & Create Copy (Fixing SettingWithCopyWarning)
        df_clean = df[['time', 'close', 'volume']].copy()
        
        # D. Add Ticker Name (Crucial for a single CSV file)
        df_clean['ticker'] = ticker
        
        # Append to list
        all_data.append(df_clean)
        
        # Sleep briefly to be nice to the API
        time.sleep(0.5)
        
    except Exception as e:
        print(f"❌ Error fetching {ticker}: {str(e)}")

# 3. Merge and Save
if all_data:
    # Combine all individual dataframes into one big table
    final_df = pd.concat(all_data, ignore_index=True)
    
    # Reorder columns to look nice: time | ticker | close | volume
    final_df = final_df[['time', 'ticker', 'close', 'volume']]
    
    # Save to CSV
    final_df.to_csv(output_file, index=False)
    
    print(f"\n✅ Success! Saved {len(final_df)} rows to '{output_file}'")
    print(final_df.head())
    print(final_df.tail())
else:
    print("\n❌ No data collected.")

Fetching data from 2014-11-26 to 2025-11-23...

Processing BIC...
Processing BMI...
Processing BVH...
Processing MIG...
Processing PGI...

✅ Success! Saved 2743 rows to '../data/raw/insurance_stocks_weekly_10y.csv'
        time ticker  close  volume
0 2014-11-30    BIC   5.15  576920
1 2014-12-07    BIC   5.38  865480
2 2014-12-14    BIC   5.42  555740
3 2014-12-21    BIC   5.11  437290
4 2014-12-28    BIC   4.92  589110
           time ticker  close  volume
2738 2025-10-26    PGI  20.00   78000
2739 2025-11-02    PGI  19.95   40400
2740 2025-11-09    PGI  19.50   26100
2741 2025-11-16    PGI  19.90    1400
2742 2025-11-23    PGI  19.70     100


In [34]:
# Read 5 insurance stocks weekly close data
df = pd.read_csv('../data/raw/insurance_stocks_weekly_10y.csv')

# Example: Create a table where columns are tickers and values are Closing Prices
df_pivot = df.pivot(index='time', columns='ticker', values='close')

print(df_pivot)
# Result:
# ticker        BIC   BMI   BVH   MIG   PGI
# time                                     
# 2014-11-30   15.5  18.2  35.5   NaN   NaN
# 2014-12-07   16.0  19.5  36.1   NaN   NaN

ticker        BIC    BMI    BVH    MIG    PGI
time                                         
2014-11-30   5.15   5.78  28.63    NaN   4.66
2014-12-07   5.38   5.68  29.33    NaN   4.66
2014-12-14   5.42   5.68  29.02    NaN   4.70
2014-12-21   5.11   5.49  24.01    NaN   4.70
2014-12-28   4.92   5.20  24.79    NaN   4.75
2015-01-04   5.07   5.36  25.03    NaN   4.99
2015-01-11   5.11   5.39  27.38    NaN   4.75
2015-01-18   5.23   5.17  27.38    NaN   4.56
2015-01-25   5.30   5.14  27.53    NaN   4.70
2015-02-01   5.26   5.14  26.98    NaN   4.75
2015-02-08   5.23   5.17  29.49    NaN   4.61
2015-02-15   5.49   5.30  31.05    NaN   4.70
2015-02-22    NaN    NaN    NaN    NaN    NaN
2015-03-01   5.64   5.30  29.33    NaN   4.70
2015-03-08   6.44   5.74  29.49    NaN   4.75
2015-03-15   6.13   5.52  29.33    NaN   4.84
2015-03-22   6.21   5.42  29.33    NaN   4.66
2015-03-29   6.46   5.39  26.91    NaN   5.04
2015-04-05   6.38   5.36  26.98    NaN   4.89
2015-04-12   6.82   5.55  26.67   

In [2]:
import pandas as pd
import numpy as np
from vnstock import Finance, Quote
from datetime import datetime, timedelta

# --- CẤU HÌNH ---
SYMBOL = 'BIC'
# Lấy dữ liệu 2 năm gần nhất để đảm bảo đủ tính MA200
END_DATE = datetime.now().strftime('%Y-%m-%d')
START_DATE = (datetime.now() - timedelta(days=730)).strftime('%Y-%m-%d')

print(f"--- ĐANG TẢI VÀ XỬ LÝ DỮ LIỆU CHO MÃ: {SYMBOL} ---")

# ==============================================================================
# PHẦN 1: DỮ LIỆU TÀI CHÍNH (FUNDAMENTAL)
# ==============================================================================
# Khởi tạo các biến mặc định
latest_ratio = pd.Series(dtype='float64')
cfo_val = np.nan
debt_to_assets = np.nan
eps_growth = np.nan

try:
    # Sử dụng nguồn VCI (Vietcap) vì có dữ liệu Ratio chi tiết nhất
    fin = Finance(symbol=SYMBOL, source='VCI')

    # 1.1 Lấy bảng chỉ số tài chính (Ratio)
    # period='quarter' để lấy dữ liệu mới nhất, lang='vi' để dễ đối chiếu
    df_ratio = fin.ratio(period='quarter', lang='vi')
    
    if not df_ratio.empty:
        # Lấy dòng mới nhất (thường là dòng đầu tiên hoặc dòng có năm/kỳ lớn nhất)
        latest_ratio = df_ratio.iloc[0]
        
        # 1.2 Tính tăng trưởng EPS (So với cùng kỳ năm ngoái - YoY)
        try:
            # Tìm dòng cùng kỳ năm ngoái (cách 4 quý)
            if len(df_ratio) >= 5:
                curr_eps = df_ratio.iloc[0].get('(Chỉ tiêu định giá, EPS (VND))', np.nan)
                prev_eps = df_ratio.iloc[4].get('(Chỉ tiêu định giá, EPS (VND))', np.nan)
                if prev_eps and prev_eps != 0:
                    eps_growth = ((curr_eps - prev_eps) / abs(prev_eps)) * 100
        except:
            pass
    
    # 1.3 Lấy dòng tiền CFO từ Báo cáo lưu chuyển tiền tệ
    df_cf = fin.cash_flow(period='quarter', lang='vi')
    if not df_cf.empty:
        # Tìm cột chứa dòng tiền HĐKD
        # Tên cột có thể khác nhau tùy loại hình DN, ta tìm từ khóa "SXKD" hoặc "hoạt động kinh doanh"
        cols = [c for c in df_cf.columns if "hoạt động kinh doanh" in str(c) or "SXKD" in str(c)]
        if cols:
            # Lấy giá trị của kỳ gần nhất (thường cột 0 là metadata, cột số liệu ở sau)
            # Trong df trả về của vnstock thường dòng là kỳ, cột là chỉ tiêu hoặc ngược lại. 
            # Check lại format mẫu: df dòng là kỳ.
            cfo_val = df_cf.iloc[0][cols[0]]

    # 1.4 Tính Total Debt / Total Assets từ Cân đối kế toán
    df_bs = fin.balance_sheet(period='quarter', lang='vi')
    if not df_bs.empty:
        # Tìm cột Tổng Tài Sản và Nợ Phải Trả
        row = df_bs.iloc[0]
        
        # Tìm cột Tổng tài sản
        asset_cols = [c for c in df_bs.columns if "TỔNG CỘNG TÀI SẢN" in str(c).upper()]
        debt_cols = [c for c in df_bs.columns if "NỢ PHẢI TRẢ" in str(c).upper()]
        
        if asset_cols and debt_cols:
            t_asset = row[asset_cols[0]]
            t_debt = row[debt_cols[0]]
            if t_asset != 0:
                debt_to_assets = t_debt / t_asset

except Exception as e:
    print(f"Lỗi khi lấy dữ liệu tài chính: {e}")

# Hàm helper để tìm giá trị trong bảng Ratio (vì tên cột VCI rất dài và phức tạp)
def get_val(keyword, fmt="{:,.2f}", scale=1):
    if latest_ratio.empty: return "Không có dữ liệu"
    for col in latest_ratio.index:
        # col có thể là tuple (Group, Indicator) hoặc string
        col_str = str(col)
        if keyword.lower() in col_str.lower():
            val = latest_ratio[col]
            if pd.isna(val): return "NaN"
            return fmt.format(val * scale)
    return "Không tìm thấy"

# ==============================================================================
# PHẦN 2: DỮ LIỆU KỸ THUẬT (TECHNICAL)
# ==============================================================================
tech_res = {}
try:
    quote = Quote(symbol=SYMBOL, source='VCI') # VCI hoặc TCBS đều được
    df_hist = quote.history(start=START_DATE, end=END_DATE)
    
    if not df_hist.empty and len(df_hist) > 50:
        close = df_hist['close']
        volume = df_hist['volume']
        high = df_hist['high']
        low = df_hist['low']

        # 1. Moving Averages
        ma20 = close.rolling(20).mean().iloc[-1]
        ma50 = close.rolling(50).mean().iloc[-1]
        ma200 = close.rolling(200).mean().iloc[-1]
        
        price = close.iloc[-1]
        
        # 2. Xu hướng
        if price > ma20 and ma20 > ma50: trend = "Tăng (Uptrend)"
        elif price < ma20 and ma20 < ma50: trend = "Giảm (Downtrend)"
        else: trend = "Đi ngang (Sideway)"
        
        # 3. MACD (12, 26, 9)
        exp12 = close.ewm(span=12, adjust=False).mean()
        exp26 = close.ewm(span=26, adjust=False).mean()
        macd_line = exp12 - exp26
        signal_line = macd_line.ewm(span=9, adjust=False).mean()
        macd_val = macd_line.iloc[-1]
        macd_sig = signal_line.iloc[-1]
        
        # 4. RSI (14)
        delta = close.diff()
        gain = (delta.where(delta > 0, 0)).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        rsi_val = rsi.iloc[-1]
        
        # 5. Stochastic (14)
        lowest_14 = low.rolling(14).min()
        highest_14 = high.rolling(14).max()
        stoch_k = 100 * ((close - lowest_14) / (highest_14 - lowest_14))
        stoch_val = stoch_k.iloc[-1]
        
        # 6. Bollinger Bands (20, 2)
        std20 = close.rolling(20).std()
        bb_up = ma20 + 2*std20.iloc[-1]
        bb_low = ma20 - 2*std20.iloc[-1]
        
        # 7. OBV
        # OBV = Previous OBV + (Volume if Price > PrevPrice else -Volume)
        obv_series = (np.sign(close.diff()) * volume).fillna(0).cumsum()
        obv_val = obv_series.iloc[-1]
        
        tech_res = {
            "price": price, "trend": trend,
            "ma20": ma20, "ma50": ma50, "ma200": ma200,
            "macd": macd_val, "macd_sig": macd_sig,
            "rsi": rsi_val, "stoch": stoch_val,
            "bb_up": bb_up, "bb_low": bb_low,
            "vol": volume.iloc[-1], "obv": obv_val
        }
    else:
        print("Không đủ dữ liệu lịch sử để tính toán kỹ thuật.")

except Exception as e:
    print(f"Lỗi tính toán kỹ thuật: {e}")


# ==============================================================================
# PHẦN 3: TỔNG HỢP BẢNG KẾT QUẢ
# ==============================================================================

# Chuẩn bị dữ liệu hiển thị
# Lưu ý: Một số hàm get_val dùng từ khóa tiếng Việt khớp với dữ liệu VCI
# EV/EBITDA và Interest Coverage đôi khi không có cho Bảo hiểm/Bank, sẽ trả về NaN hoặc Không tìm thấy

# Định dạng hiển thị
def fmt_num(val, suffix=""):
    if pd.isna(val): return "N/A"
    return f"{val:,.2f}{suffix}"

rows = [
    ["HIỆU QUẢ & CHẤT LƯỢNG SINH LỜI", "", "", ""],
    ["Biên lợi nhuận ròng", "Lãi ròng / Doanh thu thuần", "Vnstock Finance (Ratio)", get_val("Biên lợi nhuận ròng", "{:.2f}%")],
    ["ROE", "Lợi nhuận / Vốn chủ sở hữu", "Vnstock Finance (Ratio)", get_val("ROE", "{:.2f}%")],
    ["EPS", "Lãi cơ bản trên cổ phiếu", "Vnstock Finance (Ratio)", get_val("EPS (VND)", "{:,.0f} VND")],
    ["ROA", "Lợi nhuận / Tổng tài sản", "Vnstock Finance (Ratio)", get_val("ROA", "{:.2f}%")],
    ["Tăng trưởng EPS (YoY)", "% Tăng trưởng so cùng kỳ", "Tự tính từ Finance", fmt_num(eps_growth, "%")],
    ["ROIC", "Lợi nhuận / Vốn đầu tư", "Vnstock Finance (Ratio)", get_val("ROIC", "{:.2f}%")],
    ["CFO (Operating Cash Flow)", "Dòng tiền từ HĐ kinh doanh", "Vnstock Finance (CashFlow)", fmt_num(cfo_val/1e9, " tỷ")], # Chia tỷ cho gọn

    ["CẤU TRÚC TÀI CHÍNH & RỦI RO", "", "", ""],
    ["Debt / Equity", "Nợ / Vốn chủ sở hữu", "Vnstock Finance (Ratio)", get_val("Nợ/VCSH")],
    ["Total Debt / Total Assets", "Tổng nợ / Tổng tài sản", "Tự tính từ BalanceSheet", fmt_num(debt_to_assets)],
    ["Interest Coverage", "Khả năng trả lãi vay", "Vnstock Finance (Ratio)", get_val("Khả năng chi trả lãi vay")],
    ["Current Ratio", "Khả năng thanh toán hiện thời", "Vnstock Finance (Ratio)", get_val("Chỉ số thanh toán hiện thời")],

    ["ĐỊNH GIÁ", "", "", ""],
    ["P/E", "Giá / EPS", "Vnstock Finance (Ratio)", get_val("P/E")],
    ["P/B", "Giá / Giá trị sổ sách", "Vnstock Finance (Ratio)", get_val("P/B")],
    ["EV/EBITDA", "Giá trị DN / EBITDA", "Vnstock Finance (Ratio)", get_val("EV/EBITDA")],
    ["Dividend Yield", "Tỷ suất cổ tức", "Vnstock Finance (Ratio)", get_val("Tỷ suất cổ tức", "{:.2f}%")],

    ["PHÂN TÍCH KỸ THUẬT", "", "", ""],
    ["Xu hướng", "So sánh giá & MA", "Tự tính từ History", tech_res.get('trend', 'N/A')],
    ["MA20", "TB động 20 phiên", "Tự tính từ History", fmt_num(tech_res.get('ma20'), "")],
    ["MA50", "TB động 50 phiên", "Tự tính từ History", fmt_num(tech_res.get('ma50'), "")],
    ["MA200", "TB động 200 phiên", "Tự tính từ History", fmt_num(tech_res.get('ma200'), "")],
    ["MACD", "MACD Line", "Tự tính từ History", fmt_num(tech_res.get('macd'), "")],
    ["RSI", "Sức mạnh tương đối (14)", "Tự tính từ History", fmt_num(tech_res.get('rsi'), "")],
    ["Stochastic", "Dao động K% (14)", "Tự tính từ History", fmt_num(tech_res.get('stoch'), "")],
    ["Bollinger Bands", "Upper / Lower", "Tự tính từ History", f"{fmt_num(tech_res.get('bb_up'))} / {fmt_num(tech_res.get('bb_low'))}"],
    ["Volume", "Khối lượng phiên cuối", "History", fmt_num(tech_res.get('vol'), "")],
    ["OBV", "On-Balance Volume", "Tự tính từ History", fmt_num(tech_res.get('obv'), "")]
]

df_final = pd.DataFrame(rows, columns=["Chỉ tiêu", "Giải thích cơ bản", "Nơi lấy", "Giá trị / Trạng thái"])

# Hiển thị đẹp
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 50)

print("\n")
print(f"=== BẢNG TỔNG HỢP CHỈ SỐ MÃ: {SYMBOL} ===")
print(df_final)
print("\nLưu ý: Dữ liệu tài chính lấy theo Quý gần nhất được công bố.")
print("Dữ liệu kỹ thuật tính toán dựa trên giá đóng cửa phiên gần nhất.")

--- ĐANG TẢI VÀ XỬ LÝ DỮ LIỆU CHO MÃ: BIC ---


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`




=== BẢNG TỔNG HỢP CHỈ SỐ MÃ: BIC ===
                          Chỉ tiêu              Giải thích cơ bản                     Nơi lấy Giá trị / Trạng thái
0   HIỆU QUẢ & CHẤT LƯỢNG SINH LỜI                                                                                
1              Biên lợi nhuận ròng     Lãi ròng / Doanh thu thuần     Vnstock Finance (Ratio)                0.10%
2                              ROE     Lợi nhuận / Vốn chủ sở hữu     Vnstock Finance (Ratio)                0.19%
3                              EPS       Lãi cơ bản trên cổ phiếu     Vnstock Finance (Ratio)              451 VND
4                              ROA       Lợi nhuận / Tổng tài sản     Vnstock Finance (Ratio)                0.06%
5            Tăng trưởng EPS (YoY)       % Tăng trưởng so cùng kỳ          Tự tính từ Finance                  N/A
6                             ROIC         Lợi nhuận / Vốn đầu tư     Vnstock Finance (Ratio)               -0.24%
7        CFO (Operating Cash Flow)     Dò

In [3]:
import pandas as pd
from vnstock import Finance, Quote
from datetime import datetime, timedelta

# Cấu hình mã
SYMBOL = 'MIG'

print(f"--- TRÍCH XUẤT TỪNG CHỈ SỐ CHO MÃ: {SYMBOL} ---\n")

# ==============================================================================
# NHÓM 1: DỮ LIỆU THỊ TRƯỜNG (CLOSE, VOLUME)
# Nguồn: Quote.history
# ==============================================================================
print("1. LẤY CLOSE & VOLUME")

# Bước 1: Gọi API lấy lịch sử giá
quote = Quote(symbol=SYMBOL, source='VCI')
df_history = quote.history(start='2024-01-01', end=datetime.now().strftime('%Y-%m-%d'))

if not df_history.empty:
    # Lấy dòng cuối cùng (phiên gần nhất)
    last_row = df_history.iloc[-1]
    
    # --- Trích xuất Close ---
    close_price = last_row['close']
    print(f"   > Close Price: {close_price:,.2f}") # Đơn vị: 1000 VND
    
    # --- Trích xuất Volume ---
    volume = last_row['volume']
    print(f"   > Volume:      {volume:,.0f}")
else:
    print("   > Không lấy được dữ liệu lịch sử")

print("-" * 30)


# ==============================================================================
# NHÓM 2: CHỈ SỐ TÀI CHÍNH & ĐỊNH GIÁ (P/E, P/B, ROE, ROA...)
# Nguồn: Finance.ratio (Nguồn VCI có dữ liệu chi tiết nhất)
# ==============================================================================
print("2. LẤY P/E, P/B, EV/EBITDA, DIVIDEND, ROE, ROA, ROIC, EPS")

# Bước 1: Gọi API lấy bảng chỉ số (Lấy theo Quý mới nhất)
fin = Finance(symbol=SYMBOL, source='VCI')
df_ratio = fin.ratio(period='quarter', lang='vi')

if not df_ratio.empty:
    latest_data = df_ratio.iloc[0] # Dòng dữ liệu mới nhất
    
    # Hàm phụ trợ để tìm giá trị trong cột (vì tên cột VCI rất dài)
    def get_indicator(keyword):
        for col in latest_data.index:
            if keyword.lower() in str(col).lower():
                return latest_data[col]
        return None

    # --- Trích xuất P/E ---
    pe = get_indicator("P/E")
    print(f"   > P/E:         {pe:.2f}" if pe else "   > P/E: NaN")

    # --- Trích xuất P/B ---
    pb = get_indicator("P/B")
    print(f"   > P/B:         {pb:.2f}" if pb else "   > P/B: NaN")

    # --- Trích xuất EV/EBITDA ---
    ev_ebitda = get_indicator("EV/EBITDA")
    print(f"   > EV/EBITDA:   {ev_ebitda:.2f}" if ev_ebitda else "   > EV/EBITDA: NaN")
    
    # --- Trích xuất Market Cap (Vốn hóa) ---
    mkt_cap = get_indicator("Vốn hóa")
    print(f"   > Market Cap:  {mkt_cap:,.0f} tỷ" if mkt_cap else "   > Market Cap: NaN")

    # --- Trích xuất Dividend Yield (Tỷ suất cổ tức) ---
    div_yield = get_indicator("Tỷ suất cổ tức")
    print(f"   > Div Yield:   {div_yield:.2f}%" if div_yield else "   > Div Yield: 0%")

    # --- Trích xuất ROE ---
    roe = get_indicator("ROE")
    print(f"   > ROE:         {roe:.2f}%" if roe else "   > ROE: NaN")

    # --- Trích xuất ROA ---
    roa = get_indicator("ROA")
    print(f"   > ROA:         {roa:.2f}%" if roa else "   > ROA: NaN")

    # --- Trích xuất ROIC ---
    roic = get_indicator("ROIC")
    print(f"   > ROIC:        {roic:.2f}%" if roic else "   > ROIC: NaN")

    # --- Trích xuất EPS ---
    eps = get_indicator("EPS")
    print(f"   > EPS:         {eps:,.0f} VND" if eps else "   > EPS: NaN")

else:
    print("   > Không lấy được bảng chỉ số tài chính")

print("-" * 30)


# ==============================================================================
# NHÓM 3: DÒNG TIỀN (CFO)
# Nguồn: Finance.cash_flow (Báo cáo lưu chuyển tiền tệ)
# ==============================================================================
print("3. LẤY CFO (Operating Cash Flow)")

# Bước 1: Gọi API lấy báo cáo lưu chuyển tiền tệ
df_cf = fin.cash_flow(period='quarter', lang='vi')

if not df_cf.empty:
    # Bước 2: Tìm cột chứa dòng tiền kinh doanh
    # Tên cột thường chứa từ khóa "hoạt động kinh doanh" hoặc "SXKD"
    cfo_col = None
    for col in df_cf.columns:
        if "hoạt động kinh doanh" in str(col) or "SXKD" in str(col):
            cfo_col = col
            break
    
    if cfo_col:
        # Lấy giá trị kỳ gần nhất (dòng 0)
        cfo_value = df_cf.iloc[0][cfo_col]
        print(f"   > CFO:         {cfo_value:,.0f} VND")
    else:
        print("   > Không tìm thấy cột CFO trong báo cáo")
else:
    print("   > Không lấy được báo cáo lưu chuyển tiền tệ")

print("\n--- HOÀN THÀNH ---")

--- TRÍCH XUẤT TỪNG CHỈ SỐ CHO MÃ: MIG ---

1. LẤY CLOSE & VOLUME
   > Close Price: 17.00
   > Volume:      33,100
------------------------------
2. LẤY P/E, P/B, EV/EBITDA, DIVIDEND, ROE, ROA, ROIC, EPS


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


   > P/E:         8.97
   > P/B:         1.39
   > EV/EBITDA:   -6.14
   > Market Cap:  3,616,709,217,300 tỷ
   > Div Yield:   0.06%
   > ROE:         0.16%
   > ROA:         0.04%
   > ROIC:        -0.22%
   > EPS:         582 VND
------------------------------
3. LẤY CFO (Operating Cash Flow)
   > CFO:         35,715,199,977 VND

--- HOÀN THÀNH ---


In [4]:
SYMBOL = 'BIC'
fin = Finance(symbol=SYMBOL, source='VCI')
df_ratio = fin.ratio(period='quarter', lang='vi')
print(df_ratio.info())

quote = Quote(symbol=SYMBOL, source='VCI')
df_hist = quote.history(start='2022-01-01', end='2025-11-26')

print(df_hist.info())

print(df_hist.head())



Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 35 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   (Meta, CP)                                                   51 non-null     object 
 1   (Meta, Năm)                                                  51 non-null     int64  
 2   (Meta, Kỳ)                                                   51 non-null     int64  
 3   (Chỉ tiêu cơ cấu nguồn vốn, (Vay NH+DH)/VCSH)                51 non-null     float64
 4   (Chỉ tiêu cơ cấu nguồn vốn, Nợ/VCSH)                         51 non-null     float64
 5   (Chỉ tiêu cơ cấu nguồn vốn, TSCĐ / Vốn CSH)                  51 non-null     float64
 6   (Chỉ tiêu cơ cấu nguồn vốn, Vốn CSH/Vốn điều lệ)             51 non-null     float64
 7   (Chỉ tiêu hiệu quả hoạt động, Vòng quay tài sản)             51 non-null     float

In [44]:
### 0. SETUP CHUNG (BẮT BUỘC) ###
import pandas as pd
import numpy as np  # <--- Cần thêm cái này để tính OBV
from vnstock import Finance, Quote
from datetime import datetime

symbol = 'BIC'
print(f"--- ĐANG XỬ LÝ: {symbol} ---")

# Khởi tạo
fin = Finance(symbol=symbol, source='VCI')
quote = Quote(symbol=symbol, source='VCI')

# Load dữ liệu (Chạy 1 lần)
try:
    df_ratio = fin.ratio(period='quarter', lang='vi')
    df_cf = fin.cash_flow(period='quarter', lang='vi')
    df_bs = fin.balance_sheet(period='quarter', lang='vi')
    
    # Lấy ngày hiện tại
    today = datetime.now().strftime('%Y-%m-%d')
    df_hist = quote.history(start='2023-01-01', end=today) # Lấy dài ra chút để đủ tính MA200
except Exception as e:
    print(f"Lỗi load dữ liệu: {e}")

# --- HÀM GET_VAL ĐÃ SỬA (QUAN TRỌNG) ---
def get_val(keyword, source_row=None):
    """
    keyword: Từ khóa (vd 'ROE')
    source_row: Dòng dữ liệu. Mặc định là dòng đầu tiên của df_ratio (Quý mới nhất)
    """
    # Nếu không truyền dòng cụ thể, mặc định lấy dòng mới nhất từ df_ratio
    if source_row is None:
        if df_ratio.empty: return 0
        source_row = df_ratio.iloc[0]
        
    if source_row.empty: return 0
    
    for col in source_row.index:
        if keyword.lower() in str(col).lower():
            val = source_row[col]
            return val if pd.notna(val) else 0
    return 0

# ==============================================================================
# PHẦN 1: CHẠY CÁC CHỈ SỐ CƠ BẢN (FINANCIALS)
# ==============================================================================
print("\n[1] FINANCIAL INDICATORS:")

# Các chỉ số lấy trực tiếp (Giờ không cần truyền df_ratio.iloc[0] nữa)
print(f"ROE: {get_val('ROE')}")
print(f"ROA: {get_val('ROA')}")
print(f"ROIC: {get_val('ROIC')}")
print(f"Biên LN ròng: {get_val('Biên lợi nhuận ròng')}")
print(f"EPS: {get_val('EPS')}")

# Tăng trưởng EPS (Tính toán)
curr = get_val('EPS', df_ratio.iloc[0])
prev = get_val('EPS', df_ratio.iloc[4]) # Cùng kỳ năm trước
growth = (curr - prev) / abs(prev) if prev != 0 else 0
print(f"EPS Growth: {growth:.2%}")

# CFO (Tìm cột SXKD)
cfo_val = 0
if not df_cf.empty:
    # List comprehension để tìm cột
    cols = [c for c in df_cf.columns if 'SXKD' in str(c) or 'hoạt động kinh doanh' in str(c).lower()]
    if cols:
        col = cols[0]
        cfo_val = df_cf.iloc[0][col]
print(f"CFO: {cfo_val:,.0f}")

# Cấu trúc vốn
print(f"Debt/Equity: {get_val('Nợ/VCSH')}")

# Total Debt / Total Assets (Tính từ Balance Sheet)
debt_asset_ratio = 0
if not df_bs.empty:
    row_bs = df_bs.iloc[0]
    # Tìm cột nợ và tài sản (Dùng next để tìm giá trị đầu tiên khớp)
    debt = next((row_bs[c] for c in df_bs.columns if 'NỢ PHẢI TRẢ' in str(c).upper()), 0)
    asset = next((row_bs[c] for c in df_bs.columns if 'TỔNG CỘNG TÀI SẢN' in str(c).upper()), 0)
    if asset != 0:
        debt_asset_ratio = debt / asset
print(f"Total Debt/Assets: {debt_asset_ratio:.2f}")

# Các chỉ số khác
print(f"Int Coverage: {get_val('Khả năng chi trả lãi vay')}")
print(f"Current Ratio: {get_val('Chỉ số thanh toán hiện thời')}")
print(f"P/E: {get_val('P/E')}")
print(f"P/B: {get_val('P/B')}")
print(f"EV/EBITDA: {get_val('EV/EBITDA')}")
print(f"Div Yield: {get_val('Tỷ suất cổ tức')}")
print(f"Market Cap: {get_val('Vốn hóa')}")


# ==============================================================================
# PHẦN 2: CHẠY CÁC CHỈ SỐ KỸ THUẬT (TECHNICALS)
# ==============================================================================
print("\n[2] TECHNICAL INDICATORS:")

if not df_hist.empty:
    # Khai báo biến tiện lợi
    C = df_hist['close']
    
    print(f"Open: {df_hist.iloc[-1]['open']}")
    print(f"Close: {df_hist.iloc[-1]['close']}")
    print(f"Volume: {df_hist.iloc[-1]['volume']}")

    # Moving Averages
    ma20 = C.rolling(20).mean().iloc[-1]
    ma50 = C.rolling(50).mean().iloc[-1]
    ma200 = C.rolling(200).mean().iloc[-1]
    print(f"MA20: {ma20:.2f} | MA50: {ma50:.2f} | MA200: {ma200:.2f}")

    # MACD
    ema12 = C.ewm(span=12, adjust=False).mean()
    ema26 = C.ewm(span=26, adjust=False).mean()
    macd_line = ema12 - ema26
    signal_line = macd_line.ewm(span=9, adjust=False).mean()
    print(f"MACD: {macd_line.iloc[-1]:.2f} (Sig: {signal_line.iloc[-1]:.2f})")

    # RSI (Full logic)
    delta = C.diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    print(f"RSI (14): {rsi.iloc[-1]:.2f}")

    # Stochastic
    L14 = df_hist['low'].rolling(14).min()
    H14 = df_hist['high'].rolling(14).max()
    K = 100 * ((C - L14) / (H14 - L14))
    print(f"Stochastic %K: {K.iloc[-1]:.2f}")

    # Bollinger Bands
    std = C.rolling(20).std()
    upper = ma20 + 2 * std.iloc[-1]
    lower = ma20 - 2 * std.iloc[-1]
    print(f"BB Upper: {upper:.2f} | BB Lower: {lower:.2f}")

    # OBV
    sign = np.sign(C.diff())
    obv = (sign * df_hist['volume']).fillna(0).cumsum()
    print(f"OBV: {obv.iloc[-1]:,.0f}")

    # Xác nhận dòng tiền
    close = C.iloc[-1]
    prev_close = C.iloc[-2]
    vol = df_hist['volume'].iloc[-1]
    avg_vol = df_hist['volume'].rolling(20).mean().iloc[-1]
    
    if close > prev_close and vol > avg_vol:
        status = 'Tích cực (Giá tăng + Vol tăng)'
    elif close < prev_close and vol > avg_vol:
        status = 'Tiêu cực (Giá giảm + Vol tăng)'
    else:
        status = 'Trung tính'
    print(f"Dòng tiền: {status}")

else:
    print("Không có dữ liệu lịch sử giá.")

--- ĐANG XỬ LÝ: BIC ---


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



[1] FINANCIAL INDICATORS:
ROE: 0.1861441098
ROA: 0.0618952369
ROIC: -0.2442703047
Biên LN ròng: 0.09564214336637174
EPS: 450.51185280537834
EPS Growth: -25.51%
CFO: 0
Debt/Equity: 2.0494434817
Total Debt/Assets: 0.67
Int Coverage: 0
Current Ratio: 1.2678036989
P/E: 8.2755887718
P/B: 1.4991720874
EV/EBITDA: -5.6790437191
Div Yield: 0.0429184549
Market Cap: 4708162428100

[2] TECHNICAL INDICATORS:
Open: 23.15
Close: 22.5
Volume: 382100
MA20: 24.09 | MA50: 25.42 | MA200: 22.99
MACD: -0.64 (Sig: -0.60)
RSI (14): 20.29
Stochastic %K: 0.00
BB Upper: 25.56 | BB Lower: 22.63
OBV: 1,577,265
Dòng tiền: Tiêu cực (Giá giảm + Vol tăng)


In [45]:
import pandas as pd
from vnstock import Finance, Quote
from datetime import datetime

# --- CẤU HÌNH ---
SYMBOL = 'BIC'  # Thay mã cổ phiếu ở đây

print(f"--- LẤY DỮ LIỆU CỐT LÕI CHO MÃ: {SYMBOL} ---\n")

# 1. KHỞI TẠO & LOAD DỮ LIỆU
try:
    # A. Dữ liệu Tài chính (Lấy quý gần nhất)
    fin = Finance(symbol=SYMBOL, source='VCI')
    df_ratio = fin.ratio(period='quarter', lang='vi')

    # B. Dữ liệu Thị trường (Lấy phiên gần nhất)
    quote = Quote(symbol=SYMBOL, source='VCI')
    today = datetime.now().strftime('%Y-%m-%d')
    # Lấy data từ đầu năm để đảm bảo có dữ liệu, ta chỉ cần dòng cuối
    df_hist = quote.history(start='2024-01-01', end=today)

except Exception as e:
    print(f"Lỗi kết nối hoặc không tìm thấy mã: {e}")
    exit()

# 2. HÀM TRỢ GIÚP (Lấy giá trị từ tên cột phức tạp)
def get_val(df, keyword):
    if df.empty: return None
    row = df.iloc[0] # Dòng mới nhất
    for col in row.index:
        if keyword.lower() in str(col).lower():
            return row[col]
    return None

# 3. TRÍCH XUẤT VÀ IN KẾT QUẢ
print(f"{'CHỈ SỐ':<15} | {'GIÁ TRỊ':<20}")
print("-" * 40)

# --- NHÓM TÀI CHÍNH (Từ df_ratio) ---
roe = get_val(df_ratio, 'ROE')
roa = get_val(df_ratio, 'ROA')
roic = get_val(df_ratio, 'ROIC')
eps = get_val(df_ratio, 'EPS')
pe = get_val(df_ratio, 'P/E')
pb = get_val(df_ratio, 'P/B')
mkt_cap = get_val(df_ratio, 'Vốn hóa')

print(f"{'ROE':<15} | {roe:.2f}%" if roe else f"{'ROE':<15} | N/A")
print(f"{'ROA':<15} | {roa:.2f}%" if roa else f"{'ROA':<15} | N/A")
print(f"{'ROIC':<15} | {roic:.2f}%" if roic else f"{'ROIC':<15} | N/A")
print(f"{'EPS':<15} | {eps:,.0f} VND" if eps else f"{'EPS':<15} | N/A")
print(f"{'P/E':<15} | {pe:.2f}" if pe else f"{'P/E':<15} | N/A")
print(f"{'P/B':<15} | {pb:.2f}" if pb else f"{'P/B':<15} | N/A")
print(f"{'Market Cap':<15} | {mkt_cap/1e9:,.0f} tỷ" if mkt_cap else f"{'Market Cap':<15} | N/A")

# --- NHÓM THỊ TRƯỜNG (Từ df_hist) ---
if not df_hist.empty:
    last_row = df_hist.iloc[-1]
    close = last_row['close']
    volume = last_row['volume']
    
    print(f"{'Close':<15} | {close:,.2f} (nghìn đồng)")
    print(f"{'Volume':<15} | {volume:,.0f}")
else:
    print(f"{'Close':<15} | N/A")
    print(f"{'Volume':<15} | N/A")

print("-" * 40)

--- LẤY DỮ LIỆU CỐT LÕI CHO MÃ: BIC ---



Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


CHỈ SỐ          | GIÁ TRỊ             
----------------------------------------
ROE             | 0.19%
ROA             | 0.06%
ROIC            | -0.24%
EPS             | 451 VND
P/E             | 8.28
P/B             | 1.50
Market Cap      | 4,708 tỷ
Close           | 22.50 (nghìn đồng)
Volume          | 382,100
----------------------------------------


In [46]:
# --- Build 10-year weekly dataset with key ratios for HOSE insurance tickers ---
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
from vnstock import Vnstock, Finance

RAW_TICKER_PATH = Path('..') / 'data' / 'raw' / 'insurance_hose_ticket_name.csv'
OUTPUT_PATH = Path('..') / 'data' / 'processed' / 'insurance_hose_weekly_metrics_10years.csv'

# Load unique ticker symbols from the curated HOSE insurance list
insurance_symbols = (
    pd.read_csv(RAW_TICKER_PATH)["Mã CK"]
      .dropna()
      .drop_duplicates()
      .tolist()
)

start_date = (datetime.now() - timedelta(days=365 * 10)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

print(f"Collecting weekly data from {start_date} to {end_date} for: {', '.join(insurance_symbols)}")


def extract_ratio_value(row: pd.Series, keyword: str) -> float:
    """Search the VCI ratio row for the first column matching keyword."""
    if row.empty:
        return np.nan
    for col in row.index:
        if keyword.lower() in str(col).lower():
            return pd.to_numeric(row[col], errors='coerce')
    return np.nan


frames = []
for symbol in insurance_symbols:
    print(f"\n=== Processing {symbol} ===")
    stock = Vnstock().stock(symbol=symbol, source='VCI')
    hist = stock.quote.history(start=start_date, end=end_date, interval='1D')
    if hist.empty:
        print(f"⚠️  No historical quotes returned for {symbol}, skipping.")
        continue

    date_col = 'time' if 'time' in hist.columns else 'date'
    hist[date_col] = pd.to_datetime(hist[date_col])
    weekly = (
        hist.set_index(date_col)
            .resample('W')
            .agg({
                'open': 'first',
                'high': 'max',
                'low': 'min',
                'close': 'last',
                'volume': 'sum'
            })
            .dropna(subset=['close'])
            .reset_index()
    )

    fin = Finance(symbol=symbol, source='VCI')
    ratio = fin.ratio(period='quarter', lang='vi')
    latest_row = ratio.iloc[0] if not ratio.empty else pd.Series(dtype=float)

    metrics = {
        'ROE': extract_ratio_value(latest_row, 'ROE'),
        'ROA': extract_ratio_value(latest_row, 'ROA'),
        'ROIC': extract_ratio_value(latest_row, 'ROIC'),
        'EPS': extract_ratio_value(latest_row, 'EPS'),
        'P/E': extract_ratio_value(latest_row, 'P/E'),
        'P/B': extract_ratio_value(latest_row, 'P/B'),
        'Market Cap': extract_ratio_value(latest_row, 'Vốn hóa')
    }

    weekly = (
        weekly.rename(columns={date_col: 'date', 'close': 'Close', 'volume': 'Volume'})
              .assign(symbol=symbol, **metrics)
              [['date', 'symbol', 'Close', 'Volume', 'ROE', 'ROA', 'ROIC', 'EPS', 'P/E', 'P/B', 'Market Cap']]
    )

    frames.append(weekly)

if not frames:
    raise RuntimeError('No weekly data collected – check ticker list or data sources.')

weekly_metrics = (
    pd.concat(frames, ignore_index=True)
      .sort_values(['symbol', 'date'])
      .reset_index(drop=True)
)

OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
weekly_metrics.to_csv(OUTPUT_PATH, index=False)

print(f"\nSaved weekly metrics to {OUTPUT_PATH}")
weekly_metrics.head()




=== Processing BIC ===


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



=== Processing BMI ===


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



=== Processing BVH ===


...


=== Processing MIG ===


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



=== Processing PGI ===

Saved weekly metrics to ../data/processed/insurance_hose_weekly_metrics_10years.csv


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


,date,symbol,Close,Volume,ROE,ROA,ROIC,EPS,P/E,P/B,Market Cap
0,2015-06-28,BIC,6.50,137960,0.186144,0.061895,-0.24427,450.511853,8.275589,1.499172,4.708162e+12
1,2015-07-05,BIC,6.82,1118430,0.186144,0.061895,-0.24427,450.511853,8.275589,1.499172,4.708162e+12
2,2015-07-12,BIC,9.41,4124110,0.186144,0.061895,-0.24427,450.511853,8.275589,1.499172,4.708162e+12
3,2015-07-19,BIC,10.50,3356200,0.186144,0.061895,-0.24427,450.511853,8.275589,1.499172,4.708162e+12
4,2015-07-26,BIC,9.69,1881660,0.186144,0.061895,-0.24427,450.511853,8.275589,1.499172,4.708162e+12
